In [2]:
import os
from pathlib import Path
from typing import Sequence
from pylovepdf.ilovepdf import ILovePdf
from pylovepdf.tools.merge import Merge

In [3]:
_my_public_key = input("Enter your API public key: ")

## function to compress PDF

In [4]:

def compress_pdf(input_pdf: Path, compression_lvl: str = "recommended") -> None:
    """
    this function compresses the
    input PDF and writes them
    to given output path.

    Args:
    ----
        input_pdf: (Path) - input PDF path with file.
        compression_lvl: (str) - compression level
        to reduce pdf file size. Defaults to recommended.
        acceptable values are low, extreme and
        recommended.
    """
    # check input file existence:
    if not os.path.isfile(input_pdf):
        raise FileNotFoundError(f"PDF {input_pdf} doesn't exist. please check path!")
    
    # org file size:
    org_file_size = os.path.getsize(input_pdf)

    # get input file name:
    file_name = os.path.splitext(os.path.basename(input_pdf))[0]
    folder_name = os.path.dirname(input_pdf)
    
    # initialize ilovepdf:
    ilovepdf = ILovePdf(_my_public_key, verify_ssl=True)

    # create compress task:
    task = ilovepdf.new_task("compress")
    task.add_file(input_pdf)
    task.debug = False
    task.compression_level = compression_lvl
    task.set_output_folder(folder_name)

    # start task:
    task.execute()
    output_file = task.download()
    task.delete_current_task()

    # output info:
    output= os.path.join(folder_name, output_file)
    print(f"output compressed file: {output}")

    # compressed file size and compression ratio:
    cmp_file_size = os.path.getsize(os.path.join(folder_name, output_file))
    cmp_ratio = 1 - (cmp_file_size/org_file_size)

    # convert bytes to KB:
    print(f"""------
original file size: {(org_file_size/1024):.3f} KB
compressed file size: {(cmp_file_size/1024):.3f} KB
compression ratio: {cmp_ratio:.3f}""")


## function to merge pdf

In [5]:
def merge_pdf(pdf_files: Sequence[Path], merged_pdf_path: Path = './merged_output/') -> Path:
    """
    this function can be used to merge pdf
    files provided in the same sequence as 
    passed to argument pdf_files

    Args:
        pdf_files (Sequence[Path]): pdf files to be
        merged.
    Returns:
        output (Path): absolute merged output file
    """
    if len(pdf_files) < 2:
        raise ValueError("Minimum two pdf files required for merge")

    # initialize merge task:
    task = Merge(_my_public_key, verify_ssl=True, proxies=None)
    
    # merge files provided in pdf_files in same seq:
    for _file in pdf_files:
        task.add_file(_file)

    task.debug = False
    task.set_output_folder(merged_pdf_path)
    
    # start task:
    task.execute()
    output_file = task.download()
    task.delete_current_task()

    # output file:
    output = os.path.join(merged_pdf_path, output_file)

    # print output:
    print(f"merged output file: {output}")

    return output

## merge and compress function

In [9]:
def merge_and_compress_pdf(pdf_files: Sequence[Path], compression_lvl: str = "recommended") -> None:
    """
    this function can be used to
    combine pdf files and compress
    the output merged pdf

    Args:
        pdf_files (Sequence[Path]): input pdf files
        compression_lvl (str, optional): compression level for
        merged pdf. Defaults to "recommended".
    """
    _merged_file = merge_pdf(pdf_files=pdf_files)
    compress_pdf(_merged_file, compression_lvl)
    

#### Passing input pdf file from ../pdf-doc folder to create compressed pdf output with reduced size:

In [27]:
compress_pdf(input_pdf='./pdf-doc/docs-pdf/python_basics_sample_chapters.pdf', compression_lvl="extreme")

Uploading file...
File uploaded! Below file stats:
download_filename: python_basics_sample_chapters_compress_02-01-2023.pdf 
filesize: 1556490 
output_extensions: ["pdf"] 
output_filenumber: 1 
output_filesize: 690451 
status: TaskSuccess 
timer: 0.565 

File downloaded!
Task delete status: 200 

output compressed file: ./pdf-doc/docs-pdf/basics_sample_chapters_compress_02-01-2023.pdf
------
original file size: 1520.010 KB
compressed file size: 674.269 KB
compression ratio: 0.556


#### Merge two or more pdf files

In [25]:
pdf_files = ['./pdf-doc/docs-pdf/test_1.pdf', './pdf-doc/docs-pdf/test_2.pdf']

merge_pdf(pdf_files=pdf_files)

Uploading file...
File uploaded! Below file stats:
download_filename: merged_merge_02-01-2023.pdf 
filesize: 82731 
output_extensions: ["pdf"] 
output_filenumber: 1 
output_filesize: 0 
status: TaskSuccess 
timer: 0.122 

File downloaded!
Task delete status: 200 

merged output file: ./merged_output/merge_02-01-2023.pdf


#### Merge and compress pdfs

In [10]:
pdf_files = ['./pdf-doc/docs-pdf/python_basics_sample_chapters.pdf', './pdf-doc/docs-pdf/test_1.pdf', './pdf-doc/docs-pdf/test_2.pdf']

merge_and_compress_pdf(pdf_files=pdf_files, compression_lvl="extreme")

Uploading file...
File uploaded! Below file stats:
download_filename: merged_merge_03-01-2023.pdf 
filesize: 1639221 
output_extensions: ["pdf"] 
output_filenumber: 1 
output_filesize: 0 
status: TaskSuccess 
timer: 0.162 

File downloaded!
Task delete status: 200 

merged output file: ./merged_output/merge_03-01-2023.pdf
Uploading file...
File uploaded! Below file stats:
download_filename: merge_03-01-2023_compress_03-01-2023.pdf 
filesize: 1618576 
output_extensions: ["pdf"] 
output_filenumber: 1 
output_filesize: 750967 
status: TaskSuccess 
timer: 0.963 

File downloaded!
Task delete status: 200 

output compressed file: ./merged_output/03-01-2023_compress_03-01-2023.pdf
------
original file size: 1580.641 KB
compressed file size: 733.366 KB
compression ratio: 0.536
